<a href="https://colab.research.google.com/github/mathena828/GAN_Fashion-MNIST/blob/main/InfoGAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Import libraries.
import time
from math import sqrt
import tensorflow as tf
from matplotlib import pyplot
from keras.optimizers import Adam
from keras.utils import to_categorical
from numpy.random import randn, randint
from keras.models import Model, load_model
from keras.initializers import RandomNormal
from numpy import zeros, ones, expand_dims, hstack
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Activation

In [12]:
# Download the Fashion MNIST dataset.
(train_images, train_labels), (test_images, test_labels) =  tf.keras.datasets.fashion_mnist.load_data()

In [37]:
# Pre-process the 60,000 training set images.
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') # Convert pixels to floating point values. 
train_images = (train_images - 127.5) / 127.5 # Normalize the pixel values to be in the range of [-1,1].
train_dataset = train_images

In [35]:
# Perform hyperparameter optimization.
BATCH = 256
EPOCHS = 50 
LATENT_DIM = 62
N_CAT = 10
SAMPLES = 100

In [7]:
# Define the generator.
def build_generator(input_size):
	init = RandomNormal(stddev=0.02) # Initialize with random Gaussian weights.
	input_latent = Input(shape=(input_size,))
  # Add a fully connected layer to take the input vector and produce a some numbers of activations to create 512 7×7.
	nodes = 512 * 7 * 7
	gen = Dense(nodes, kernel_initializer=init)(input_latent) 
	gen = Activation('relu')(gen) 
	gen = BatchNormalization()(gen)
	gen = Reshape((7, 7, 512))(gen)
  # Upsample with normal convolution layers.
	gen = Conv2D(128, (4,4), padding='same', kernel_initializer=init)(gen)
	gen = Activation('relu')(gen)
	gen = BatchNormalization()(gen)
	gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
	gen = Activation('relu')(gen)
	gen = BatchNormalization()(gen)
	gen = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
	output_layer = Activation('tanh')(gen)
	model = Model(input_latent, output_layer) # Define the model.
	return model

In [10]:
# Build the generator.
input_size = LATENT_DIM + N_CAT
generator = build_generator(input_size)

In [14]:
# Define the discriminator.
def build_discriminator(n_cat, input_shape=(28,28,1)):
	init = RandomNormal(stddev=0.02)
	input_image = Input(shape=input_shape)
	# Downsample using Conv2D and LeakyReLU.
	disc = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(input_image)
	disc = LeakyReLU(alpha=0.1)(disc)
	disc = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(disc)
	disc = LeakyReLU(alpha=0.1)(disc)
	disc = BatchNormalization()(disc)
	disc = Conv2D(256, (4,4), padding='same', kernel_initializer=init)(disc)
	disc = LeakyReLU(alpha=0.1)(disc)
	disc = BatchNormalization()(disc)
	disc = Flatten()(disc) # Flatten the feature maps. 
	classifier = Dense(1, activation='sigmoid')(disc) 	# Predict the probability of an output being real or fake with the sigmoid activation function.
	# Define and compile the model with an Adam optimizer.
	discriminator = Model(input_image, classifier)
	discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
	# Create the auxiliary model layers.
	aux = Dense(128)(disc)
	aux = BatchNormalization()(aux)
	aux = LeakyReLU(alpha=0.1)(aux)
	codes = Dense(n_cat, activation='softmax')(aux) # Predict the control variable using the softmax activation function.
	auxiliary = Model(input_image, codes) # Define the model.
	return discriminator, auxiliary

In [16]:
# Build the discriminator.
discriminator, auxiliary = build_discriminator(N_CAT)

In [21]:
# Define the InfoGAN.
def build_infogan(generator, discriminator, auxiliary):
	discriminator.trainable = False
	# Connect the output of the generator to the input of the discriminator and auxiliary models.
	discriminator_output = discriminator(generator.output)
	auxiliary_output = auxiliary(generator.output)
	# Define the composite model with an Adam optimizer.
	model = Model(generator.input, [discriminator_output, auxiliary_output])
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=opt)
	return model

In [28]:
# Build the InfoGAN.
infogan = build_infogan(generator, discriminator, auxiliary)

In [44]:
# Produce points in the latent space as input for the generator.
def get_latent_points(latent_dim, n_cat, samples):
	z_latent = randn(latent_dim * samples)
	z_latent = z_latent.reshape(samples, latent_dim)
	codes = randint(0, n_cat, samples)
	codes = to_categorical(codes, num_classes=n_cat)
	z_input = hstack((z_latent, codes)) # Concatenate the latent points and control codes.
	return [z_input, codes]

In [41]:
# Select real samples from the dataset.
def get_real_samples(train_dataset, samples):
	index = randint(0, train_dataset.shape[0], samples)
	x = train_dataset[index]
	y = ones((samples, 1))  # Generate class labels of value 1 to indicate the images are real.
	return x, y

In [42]:
# Generate fake images with class labels using the generator.
def get_fake_samples(generator, latent_dim, n_cat, samples):
	z_input, _ = get_latent_points(latent_dim, n_cat, samples)
	images = generator.predict(z_input)
	y = zeros((samples, 1)) # Generate class labels of value 0 to indicate the images are fake.
	return images, y

In [32]:
# Periodically use the generator to generate a sample of images and save the generator and composite models to a file.
def summarize_performance(step, generator, infogan, latent_dim, n_cat, samples=100):
	fakes, _ = get_fake_samples(generator, latent_dim, n_cat, samples)
	fakes = (fakes + 1) / 2.0 	# Scale from [-1,1] to [0,1]
  # Plot the images.
	for i in range(100):
		pyplot.subplot(10, 10, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X[i, :, :, 0], cmap='gray')
	# Save the plot to a file.
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# Save the generator.
	filename2 = 'generator_%04d.h5' % (step+1)
	generator.save(filename2)
	# Save the InfoGAN.
	filename3 = 'infogan_%04d.h5' % (step+1)
	gan_model.save(filename3)
	print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

In [49]:
# Iterate over the number of epochs and train the InfoGAN on the dataset in batches.
def train(generator, discriminator, infogan, train_dataset, latent_dim, n_cat, epochs, batch):
  batch_per_epoch = int(train_dataset.shape[0] / batch)
  steps = batch_per_epoch * epochs
  half_batch = int(batch / 2)
  start = time.time() # Get the starting time of each epoch.
  for step in range(steps):
    # Update the discriminator and auxiliary model weights using randomly selected real images.
    x_real, y_real = get_real_samples(train_dataset, half_batch)
    d_loss1 = discriminator.train_on_batch(x_real, y_real)
    # Update discriminator model weights using fake images.
    x_fake, y_fake = get_fake_samples(generator, latent_dim, n_cat, half_batch)
    d_loss2 = discriminator.train_on_batch(x_fake, y_fake)
    # Update the generator based on the the discriminator and auxiliary models' errors.
    z_input, codes = get_latent_points(latent_dim, n_cat, batch)
    y_infogan = ones((batch, 1))
    _,g_1,g_2 = infogan.train_on_batch(z_input, [y_infogan, codes])
    # print('>%d, d[%.3f,%.3f], g[%.3f] q[%.3f]' % (step+1, d_loss1, d_loss2, g_1, g_2))
    # Evaluate the model's performance during every epoch.
    if (step+1) % (batch_per_epoch * 10) == 0:
      summarize_performance(step, generator, infogan, latent_dim, n_cat)
      print('Time for epoch {} is {} seconds.'.format(epoch + 1, time.time()-start)) # Record how long it took for each epoch to run.
      start = time.time() # Get the starting time of each epoch.

In [ ]:
train(generator, discriminator, infogan, train_dataset, LATENT_DIM, N_CAT, EPOCHS, BATCH)

In [ ]:
# Create a plot of generated images
def create_plot(examples, n_examples):
	for i in range(n_examples):
		pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)
		pyplot.axis('off')
		pyplot.imshow(examples[i, :, :, 0], cmap='gray')
	pyplot.show()
 
# Load the trained model.
model = load_model('model.h5', compile=False)
CATEGORY = 9 # Define the category that will be evaluated.
z_input, _ = get_latent_points(LATENT_DIM, N_CAT, SAMPLES, CATEGORY)
X = model.predict(z_input)
X = (X + 1) / 2.0 # Scale from [-1,1] to [0,1]
create_plot(X, SAMPLES)